# tensorflowの場合

In [1]:
import tensorflow as tf

# 1. 環境の準備とデータの読み込み
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# 2. データの前処理
x_train, x_test = x_train / 255.0, x_test / 255.0

# 3. モデルの構築
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')  # CIFAR-10は10クラスのため出力層は10ノード
])

# 4. モデルのコンパイル
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 5. モデルのトレーニング
history = model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

# 6. モデルの評価
test_loss, test_acc = model.evaluate(x_test,  y_test, verbose=2)
print(f'Test accuracy: {test_acc}')


170498071/170498071 [==============================] - 2s 0us/step
Epoch 1/10
1563/1563 [==============================] - 13s 6ms/step - loss: 1.5425 - accuracy: 0.4366 - val_loss: 1.2888 - val_accuracy: 0.5341
Epoch 2/10
1563/1563 [==============================] - 8s 5ms/step - loss: 1.2026 - accuracy: 0.5706 - val_loss: 1.1433 - val_accuracy: 0.5880
Epoch 3/10
1563/1563 [==============================] - 7s 5ms/step - loss: 1.0498 - accuracy: 0.6297 - val_loss: 1.0324 - val_accuracy: 0.6332
Epoch 4/10
1563/1563 [==============================] - 8s 5ms/step - loss: 0.9520 - accuracy: 0.6672 - val_loss: 0.9938 - val_accuracy: 0.6492
Epoch 5/10
1563/1563 [==============================] - 8s 5ms/step - loss: 0.8805 - accuracy: 0.6923 - val_loss: 0.9179 - val_accuracy: 0.6798
Epoch 6/10
1563/1563 [==============================] - 8s 5ms/step - loss: 0.8149 - accuracy: 0.7143 - val_loss: 0.8947 - val_accuracy: 0.6917
Epoch 7/10
1563/1563 [==============================] - 8s 5ms/step 

# Pytorchの場合

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 1. 環境の準備とデータの読み込み
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)

testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

# 2. モデルの構築
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.conv3 = nn.Conv2d(64, 64, 3)
        self.fc1 = nn.Linear(64 * 4 * 4, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = x.view(-1, 64 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

net = Net()

# 3. 損失関数とオプティマイザの設定
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

# 4. モデルのトレーニング
for epoch in range(10):

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 200 == 199:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')

# 5. モデルの評価
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Test accuracy: {100 * correct // total}%')

100%|██████████| 170498071/170498071 [00:01<00:00, 105654195.79it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
[1,   200] loss: 1.869
[1,   400] loss: 1.564
[1,   600] loss: 1.416
[2,   200] loss: 1.269
[2,   400] loss: 1.208
[2,   600] loss: 1.164
[3,   200] loss: 1.066
[3,   400] loss: 1.042
[3,   600] loss: 1.010
[4,   200] loss: 0.925
[4,   400] loss: 0.951
[4,   600] loss: 0.907
[5,   200] loss: 0.842
[5,   400] loss: 0.848
[5,   600] loss: 0.823
[6,   200] loss: 0.772
[6,   400] loss: 0.779
[6,   600] loss: 0.760
[7,   200] loss: 0.712
[7,   400] loss: 0.708
[7,   600] loss: 0.739
[8,   200] loss: 0.652
[8,   400] loss: 0.663
[8,   600] loss: 0.683
[9,   200] loss: 0.594
[9,   400] loss: 0.629
[9,   600] loss: 0.648
[10,   200] loss: 0.565
[10,   400] loss: 0.574
[10,   600] loss: 0.601
Finished Training
Test accuracy: 72%
